# L3 M11.2: Tenant Metadata & Registry Design

**Learning Arc:**

In M11.1, you built tenant routing middleware and isolation patterns. Now you're adding the critical missing piece: the **Tenant Registry** - your single source of truth for all tenant metadata, lifecycle management, and feature configuration.

**Problem:**
- 15+ tenants with scattered configuration across environment variables
- No single source of truth for tenant metadata
- Manual multi-step process to add/suspend tenants
- No audit trail or compliance controls

**Solution:**

Build a production-ready tenant registry with:
- PostgreSQL schema (20+ attributes per tenant)
- Feature flag service (hierarchical evaluation)
- Lifecycle state machine (valid transitions only)
- Cascading operations (atomic multi-system changes)
- Health monitoring (tenant health scores)

**In this notebook (12 sections):**
1. PostgreSQL schema design & tenant registry initialization
2. Tenant CRUD operations (create, read, update, list)
3. Feature flag hierarchical evaluation
4. Lifecycle state machine transitions
5. Cascading operations across systems
6. Health score calculation
7. Reality check: Feature flag explosion at scale
8. Alternative solutions comparison
9. Common failure scenarios & prevention
10. GCC enterprise context
11. Decision card & cost-benefit analysis
12. PractaThon mission & summary

Let's build!

## Section 1: Setup and Imports

In [ ]:
import os
import sys

# Add parent directory to path for imports
sys.path.insert(0, os.path.abspath('..'))

# Set logging to INFO
os.environ["LOG_LEVEL"] = "INFO"

from src.l3_m11_multi_tenant_foundations import (
    TenantRegistry, TenantStatus, Tenant,
    FeatureFlagService, FeatureFlag,
    LifecycleManager, CascadingOperations, HealthMonitor
)

print("✅ Imports successful")
print("📝 Note: This module demonstrates tenant registry patterns")
print("   Live database connections are optional for learning")

# SAVED_SECTION:1

## Section 2: Initialize Tenant Registry

The TenantRegistry is our central coordinator integrating:
- Tenant metadata storage
- Feature flag service
- Lifecycle manager
- Cascading operations
- Health monitor

In [ ]:
# Initialize tenant registry
registry = TenantRegistry()

print("✅ Tenant Registry initialized")
print(f"Components: {list(registry.__dict__.keys())}")
# Expected: 5 integrated services (tenants, feature_service, lifecycle_manager, cascade_ops, health_monitor)

# SAVED_SECTION:2

## Section 3: Create Multiple Tenants

Let's create 5 tenants representing different business units in a GCC:
- 1 platinum tenant (Finance)
- 2 gold tenants (Legal, Global Operations)
- 2 silver tenants (HR, Marketing)

In [ ]:
tenants_config = [
    {
        "tenant_id": "finance",
        "tenant_name": "Finance Department",
        "tier": "platinum",
        "max_users": 100,
        "max_documents": 50000,
        "max_queries_per_day": 10000,
        "sla_target": 0.9999,
        "support_level": "24/7"
    },
    {
        "tenant_id": "legal",
        "tenant_name": "Legal Department",
        "tier": "platinum",
        "max_users": 50,
        "max_documents": 10000,
        "max_queries_per_day": 5000,
        "sla_target": 0.9999,
        "support_level": "24/7"
    },
    {
        "tenant_id": "hr",
        "tenant_name": "HR Operations",
        "tier": "gold",
        "max_users": 30,
        "max_documents": 5000,
        "max_queries_per_day": 2000,
        "sla_target": 0.999,
        "support_level": "business-hours"
    },
    {
        "tenant_id": "ops",
        "tenant_name": "Global Operations",
        "tier": "gold",
        "max_users": 50,
        "max_documents": 8000,
        "max_queries_per_day": 3000,
        "sla_target": 0.999,
        "support_level": "business-hours"
    },
    {
        "tenant_id": "marketing",
        "tenant_name": "Marketing",
        "tier": "silver",
        "max_users": 20,
        "max_documents": 2000,
        "max_queries_per_day": 500,
        "sla_target": 0.99,
        "support_level": "email-only"
    }
]

for config in tenants_config:
    tenant = registry.create_tenant(config)
    print(f"Created: {tenant.tenant_name} ({tenant.tier} tier) - {tenant.max_users} users")

print(f"\n✅ Total tenants: {len(registry.tenants)}")
# Expected: 5 tenants created

# SAVED_SECTION:3

## Section 4: Retrieve and List Tenants

Demonstrate CRUD operations:
- Get tenant by ID
- List all tenants
- Filter by status, tier, health score

In [ ]:
# Get specific tenant
finance = registry.get_tenant("finance")
print("Finance Department:")
print(f"  Tier: {finance.tier}")
print(f"  Status: {finance.status.value}")
print(f"  Health: {finance.health_score}/100")
print(f"  SLA Target: {finance.sla_target * 100}%")
# Expected: platinum tier, active status, 100 health

print("\n" + "="*50)
print("All Active Tenants:")
print("="*50)
active_tenants = registry.list_tenants(status=TenantStatus.ACTIVE)
for t in active_tenants:
    print(f"  - {t.tenant_name:30} {t.tier:10} {t.max_users} users")
# Expected: All 5 tenants listed

print("\n" + "="*50)
print("Platinum Tier Tenants:")
print("="*50)
platinum = registry.list_tenants(tier="platinum")
for t in platinum:
    print(f"  - {t.tenant_name}")
# Expected: Finance, Legal

# SAVED_SECTION:4

## Section 5: Feature Flag Hierarchical Evaluation

Hierarchy: **Tenant Override > Tier Default > Global Default**

We'll configure:
- Global defaults (all tenants)
- Tier defaults (platinum/gold/silver)
- Tenant overrides (specific tenant customization)

In [ ]:
# Set global defaults
registry.feature_service.global_defaults["basic_search"] = True
registry.feature_service.global_defaults["advanced_analytics"] = False
registry.feature_service.global_defaults["api_access"] = False

# Set tier defaults
registry.feature_service.tier_defaults = {
    "platinum": {
        "advanced_analytics": True,
        "dedicated_namespace": True,
        "priority_support": True,
        "api_access": True
    },
    "gold": {
        "advanced_analytics": True,
        "shared_namespace": True,
        "api_access": True
    },
    "silver": {
        "advanced_analytics": False,
        "shared_namespace": True,
        "api_access": False
    }
}

# Set tenant override (Marketing gets advanced_analytics despite being silver)
registry.feature_service.tenant_flags["marketing"] = {"advanced_analytics": True, "api_access": True}

print("Feature Flag Evaluation:")
print("="*60)

# Test hierarchy
print("\nFeature: advanced_analytics")
finance_result = registry.feature_service.evaluate("finance", "advanced_analytics", "platinum")
print(f"  Finance (platinum):           {finance_result} ← tier default")

hr_result = registry.feature_service.evaluate("hr", "advanced_analytics", "gold")
print(f"  HR (gold):                     {hr_result} ← tier default")

marketing_result = registry.feature_service.evaluate("marketing", "advanced_analytics", "silver")
print(f"  Marketing (silver + override): {marketing_result} ← tenant override wins!")

print("\nFeature: dedicated_namespace")
finance_ns = registry.feature_service.evaluate("finance", "dedicated_namespace", "platinum")
print(f"  Finance (platinum):            {finance_ns} ← tier default")

hr_ns = registry.feature_service.evaluate("hr", "dedicated_namespace", "gold")
print(f"  HR (gold):                     {hr_ns} ← global default (False)")

# Expected: Hierarchy works correctly

# SAVED_SECTION:5

## Section 6: Lifecycle State Machine

Valid transitions:
- `ACTIVE` ↔ `SUSPENDED` (temporary suspension)
- `ACTIVE` → `MIGRATING` (data migration)
- `MIGRATING` → `ACTIVE` or `SUSPENDED`
- `SUSPENDED` → `ARCHIVED` (long-term retention)
- `ARCHIVED` → `DELETED` (final deletion)

In [ ]:
print("Lifecycle State Machine Demonstrations:")
print("="*60)

# Valid transition: ACTIVE -> SUSPENDED
print("\n1. Valid: ACTIVE → SUSPENDED")
hr_tenant = registry.get_tenant("hr")
print(f"   Before: {hr_tenant.status.value}")
registry.lifecycle_manager.transition(hr_tenant, TenantStatus.SUSPENDED, "Budget review")
print(f"   After:  {hr_tenant.status.value} ✅")

# Valid transition: SUSPENDED -> ACTIVE
print("\n2. Valid: SUSPENDED → ACTIVE")
print(f"   Before: {hr_tenant.status.value}")
registry.lifecycle_manager.transition(hr_tenant, TenantStatus.ACTIVE, "Budget approved")
print(f"   After:  {hr_tenant.status.value} ✅")

# Invalid transition: ACTIVE -> DELETED
print("\n3. Invalid: ACTIVE → DELETED (should fail)")
marketing_tenant = registry.get_tenant("marketing")
try:
    registry.lifecycle_manager.transition(marketing_tenant, TenantStatus.DELETED)
    print("   ❌ ERROR: Should have failed!")
except ValueError as e:
    print(f"   ✅ Rejected: {e}")

# Valid multi-step transition: ACTIVE -> SUSPENDED -> ARCHIVED
print("\n4. Valid path: ACTIVE → SUSPENDED → ARCHIVED")
legal_tenant = registry.get_tenant("legal")
print(f"   Start: {legal_tenant.status.value}")
registry.lifecycle_manager.transition(legal_tenant, TenantStatus.SUSPENDED)
print(f"   Step 1: {legal_tenant.status.value}")
registry.lifecycle_manager.transition(legal_tenant, TenantStatus.ARCHIVED)
print(f"   Step 2: {legal_tenant.status.value} ✅")

# Get valid transitions for current state
print("\n5. Valid transitions from ARCHIVED:")
valid_next = registry.lifecycle_manager.get_valid_transitions(legal_tenant.status)
print(f"   {valid_next}")
# Expected: ['deleted'] only

# SAVED_SECTION:6

## Section 7: Cascading Operations

When suspending/activating a tenant, changes propagate atomically across:
- PostgreSQL (RLS policies)
- Vector DB (namespace access)
- S3 (bucket permissions)
- Redis (cache invalidation)
- Monitoring (alert rules)

In [ ]:
print("Cascading Tenant Suspension:")
print("="*60)

# Suspend Finance tenant
print("\nSuspending Finance tenant across all systems...")
result = registry.suspend_tenant("finance", reason="Monthly maintenance")

print(f"\nTenant: {result['tenant_id']}")
print(f"Status: {result['status']}")
print(f"\nOperations completed:")
for system, status in result['operations'].items():
    print(f"  {system:15} {status['status']}")

# Expected: 5 systems updated (postgresql, vector_db, s3, redis, monitoring)

print("\n" + "="*60)
print("Cascading Tenant Activation:")
print("="*60)

# Activate Finance tenant
print("\nActivating Finance tenant across all systems...")
result = registry.activate_tenant("finance", reason="Maintenance complete")

print(f"\nTenant: {result['tenant_id']}")
print(f"Status: {result['status']}")
print(f"\nOperations completed:")
for system, status in result['operations'].items():
    print(f"  {system:15} {status['status']}")

# SAVED_SECTION:7

## Section 8: Health Score Calculation

Calculate tenant health scores (0-100) from multiple signals:
- API latency (P95)
- Error rate
- Query success rate
- Storage utilization

Scoring penalties:
- High latency (>500ms): -10 points per 100ms
- Error rate: -20 points per 1%
- Low query success (<95%): -30 points
- High storage (>90%): -20 points

In [ ]:
print("Tenant Health Score Calculation:")
print("="*60)

# Scenario 1: Healthy tenant (Finance)
print("\n1. Finance (Healthy):")
finance_score = registry.health_monitor.calculate_health_score(
    tenant_id="finance",
    latency_p95=350.0,  # Under 500ms
    error_rate=0.005,   # 0.5% errors
    query_success_rate=0.98,  # 98% success
    storage_utilization=0.70  # 70% storage
)
finance_tenant = registry.get_tenant("finance")
finance_tenant.update_health_score(finance_score)
print(f"   Latency P95: 350ms | Error Rate: 0.5% | Success: 98% | Storage: 70%")
print(f"   Health Score: {finance_score}/100 ✅")

# Scenario 2: Degraded tenant (HR)
print("\n2. HR (Degraded):")
hr_score = registry.health_monitor.calculate_health_score(
    tenant_id="hr",
    latency_p95=700.0,  # 200ms over threshold
    error_rate=0.02,    # 2% errors
    query_success_rate=0.93,  # 93% success (below 95%)
    storage_utilization=0.85  # 85% storage
)
hr_tenant = registry.get_tenant("hr")
hr_tenant.update_health_score(hr_score)
print(f"   Latency P95: 700ms | Error Rate: 2% | Success: 93% | Storage: 85%")
print(f"   Health Score: {hr_score}/100 ⚠️")

# Scenario 3: Critical tenant (Marketing)
print("\n3. Marketing (Critical):")
mkt_score = registry.health_monitor.calculate_health_score(
    tenant_id="marketing",
    latency_p95=1200.0,  # Very high latency
    error_rate=0.10,     # 10% errors
    query_success_rate=0.80,  # 80% success
    storage_utilization=0.95  # 95% storage (over 90%)
)
mkt_tenant = registry.get_tenant("marketing")
mkt_tenant.update_health_score(mkt_score)
print(f"   Latency P95: 1200ms | Error Rate: 10% | Success: 80% | Storage: 95%")
print(f"   Health Score: {mkt_score}/100 🚨")

# Show health metrics
print("\n" + "="*60)
print("Cached Health Metrics:")
print("="*60)
for tenant_id in ["finance", "hr", "marketing"]:
    metrics = registry.health_monitor.get_metrics(tenant_id)
    if metrics:
        print(f"\n{tenant_id.upper()}:")
        print(f"  Health Score: {metrics['health_score']}/100")
        print(f"  Latency P95:  {metrics['latency_p95']}ms")
        print(f"  Error Rate:   {metrics['error_rate']*100:.1f}%")

# SAVED_SECTION:8

## Section 9: Reality Check - Feature Flag Explosion

**From Script Part 2, Section 5: The Myth vs. Reality**

### The Myth
"Feature flags solve everything - just put every tenant difference behind a flag!"

### The Reality

**Flag Explosion Timeline:**
- Month 1: 20 flags (manageable)
- Month 6: 150 flags (getting messy)
- Month 18: 500+ flags (complete chaos)

**Problems at Scale:**
1. **Performance**: Evaluating 500 flags adds 250ms latency
2. **Testing**: 2^500 possible combinations (impossible to test)
3. **Dependency Hell**: Flag A requires B, B conflicts with C
4. **Cost**: ₹15,000/month just evaluating flags

### The Solution

**Flags are for TEMPORARY rollouts, not PERMANENT configuration:**

✅ **Good**: Rolling out new feature to 10% → 50% → 100% over 2 weeks
❌ **Bad**: Using flags for max_users limits (that's config, not a flag)

**Rule:** If a flag will exist >90 days, move it to database configuration.

**Savings:** This discipline saved ₹12 lakh/year at one GCC platform.

In [ ]:
print("Reality Check: Feature Flags at Scale")
print("="*60)
print("\n⚠️  WARNING: Avoid flag explosion!")
print("\nBad Practice:")
print("  ❌ 500+ flags for permanent configuration")
print("  ❌ Flags living >90 days")
print("  ❌ Using flags for tenant limits/quotas")
print("\nGood Practice:")
print("  ✅ <50 flags per tenant")
print("  ✅ Flags for temporary rollouts only")
print("  ✅ Use tier configs for stable differences")
print("  ✅ Remove deprecated flags aggressively")

print("\nCost Impact:")
print("  500 flags × 0.5ms evaluation × 1M requests/day")
print("  = 250ms added latency")
print("  = ₹15,000/month wasted on flag evaluation")

print("\nBetter Approach:")
print("  Use tier_defaults table for permanent differences")
print("  Keep flags under 50 per tenant")
print("  Savings: ₹12 lakh/year")

# SAVED_SECTION:9

## Section 10: Alternative Solutions Comparison

**From Script Part 2, Section 6: When to Use What**

### Four Approaches:

1. **Manual Config Files** - Best for <5 tenants
2. **Centralized Registry (This Module)** - Best for 10-100 tenants
3. **Service Mesh (Istio)** - Best for 50+ tenants
4. **Managed SaaS (Frontegg)** - Best for <20 tenants, limited customization

### Cost Comparison:

| Approach | Annual Cost | Scalability | Complexity |
|----------|-------------|-------------|------------|
| Config Files | ₹0 (dev time) | Poor (<5 tenants) | Low |
| Centralized DB | ₹11L/year | Good (10-100) | Medium |
| Service Mesh | ₹24.6L/year | Excellent (50+) | High |
| Managed SaaS | $6K-60K/year | Good (<20) | Low |

In [ ]:
print("Alternative Solutions Comparison")
print("="*60)

alternatives = [
    {
        "name": "Config Files (YAML/JSON)",
        "best_for": "<5 tenants",
        "cost": "₹0 infrastructure",
        "pros": "Simple, version controlled",
        "cons": "Requires deployment for changes"
    },
    {
        "name": "Centralized Registry (This Module)",
        "best_for": "10-100 tenants",
        "cost": "₹11L/year",
        "pros": "Single source of truth, runtime updates",
        "cons": "Database becomes critical dependency"
    },
    {
        "name": "Service Mesh (Istio)",
        "best_for": "50+ tenants",
        "cost": "₹24.6L/year",
        "pros": "Infrastructure-level routing, excellent scale",
        "cons": "High complexity, steep learning curve"
    },
    {
        "name": "Managed SaaS (Frontegg, Auth0)",
        "best_for": "<20 tenants",
        "cost": "$6K-60K/year",
        "pros": "Fast setup, vendor maintains",
        "cons": "Vendor lock-in, expensive at scale"
    }
]

for i, alt in enumerate(alternatives, 1):
    print(f"\n{i}. {alt['name']}")
    print(f"   Best For: {alt['best_for']}")
    print(f"   Cost:     {alt['cost']}")
    print(f"   Pros:     {alt['pros']}")
    print(f"   Cons:     {alt['cons']}")

print("\n" + "="*60)
print("Recommendation for GCC (20-100 business units):")
print("="*60)
print("✅ Use Centralized Database Registry (this module)")
print("\nWhy?")
print("  • Saves ₹7L/year vs. manual configuration")
print("  • Meets compliance requirements (audit logs)")
print("  • Scales to 100 tenants without redesign")
print("  • Runtime updates without deployment")

# SAVED_SECTION:10

## Section 11: Common Failures & Prevention

**From Script Part 2, Section 8: Production Failure Scenarios**

### Top 5 Failures:

1. **Database Outage** → Platform offline for 47 minutes → ₹5L penalty
   - **Prevention**: PostgreSQL replicas + Redis fallback + disk backups

2. **Cache Inconsistency** → Suspended tenant still active → ₹48K cost
   - **Prevention**: Active cache invalidation + verification

3. **Stale Limits** → False alerts after tier upgrade
   - **Prevention**: Tiered cache TTL (critical=1min, standard=5min)

4. **Permission Bypass** → Cross-tenant data leak
   - **Prevention**: Strict authorization checks + audit logging

5. **Audit Log Tampering** → Compliance violation
   - **Prevention**: Immutable audit logs + cryptographic signatures

In [ ]:
print("Common Failures & Prevention Strategies")
print("="*60)

failures = [
    {
        "name": "Database Outage",
        "impact": "Platform offline 47min → ₹5L penalty",
        "prevention": "PG replicas + Redis fallback + backups"
    },
    {
        "name": "Cache Inconsistency",
        "impact": "Suspended tenant active 5min → ₹48K cost",
        "prevention": "Active cache invalidation + verification"
    },
    {
        "name": "Stale Tenant Limits",
        "impact": "False alerts after tier upgrade",
        "prevention": "Tiered TTL: critical=1min, standard=5min"
    },
    {
        "name": "Permission Bypass",
        "impact": "Cross-tenant data leak (GDPR violation)",
        "prevention": "Strict authz checks + audit all API calls"
    },
    {
        "name": "Audit Log Tampering",
        "impact": "Compliance violation (no proof of changes)",
        "prevention": "Immutable logs + crypto signatures"
    }
]

for i, failure in enumerate(failures, 1):
    print(f"\n{i}. {failure['name']}")
    print(f"   Impact:     {failure['impact']}")
    print(f"   Prevention: {failure['prevention']}")

print("\n" + "="*60)
print("Key Takeaway:")
print("="*60)
print("Layer defenses - assume components WILL fail.")
print("Verify critical operations twice.")
print("Treat audit logs as immutable compliance artifacts.")

# SAVED_SECTION:11

## Section 12: GCC Enterprise Context & Decision Card

**From Script Part 3, Sections 9C-10: Production Considerations**

### What's Different in GCC Multi-Tenancy?

**1. Tenant = Business Unit (not external customer)**
- Legal: 50 attorneys, 10K privileged docs, attorney-client privilege
- Finance: 100 analysts, real-time trading data, SOX compliance
- HR: 30 recruiters, 50K employee records, GDPR Article 9

**2. Shared Users Across Tenants**
- Same employee works in multiple departments
- Role-based access per tenant (Alice: Finance Analyst + Legal Reviewer)

**3. Cost Attribution & Chargeback**
- Legal: ₹2.48L/month → Budget ₹2.5L → 0.8% under ✅
- Finance: ₹4.7L/month → Budget ₹4.2L → 11.9% over ⚠️

**4. Three-Layer Compliance**
- Parent company policies
- India regulations (IT Act, DPDP)
- Global standards (GDPR, SOX, SEC)

### Decision Card: When to Use Tenant Registry

**Use When:**
✅ 20+ tenants with different requirements
✅ Need feature flag management at scale
✅ Compliance requires audit trails
✅ Multi-tier service offerings
✅ Chargeback/cost attribution needed

**Avoid When:**
❌ <5 tenants (manual config sufficient)
❌ All tenants identical (no customization)
❌ Using managed SaaS (Frontegg handles it)

**Break-Even Analysis:**
- Small GCC (20 tenants): Month 8 in Year 1
- Medium GCC (50 tenants): Month 6 in Year 1
- Large GCC (100 tenants): Month 9 in Year 1

In [ ]:
print("GCC Enterprise Context & Decision Card")
print("="*60)

# GCC Multi-Tenancy Differences
print("\n🏢 GCC Multi-Tenancy Characteristics:")
print("  • Tenant = Business Unit (not external customer)")
print("  • Shared users across tenants (role-based access)")
print("  • Cost attribution & chargeback to departments")
print("  • Three-layer compliance (Parent/India/Global)")

# Cost Analysis
print("\n💰 Cost Analysis by GCC Size:")
gcc_sizes = [
    {"size": "Small (20 tenants)", "monthly": "₹45K", "annual": "₹5.4L", "breakeven": "Month 8"},
    {"size": "Medium (50 tenants)", "monthly": "₹1.25L", "annual": "₹15L", "breakeven": "Month 6"},
    {"size": "Large (100 tenants)", "monthly": "₹3.6L", "annual": "₹43.2L", "breakeven": "Month 9"}
]

for gcc in gcc_sizes:
    print(f"\n  {gcc['size']}:")
    print(f"    Monthly Cost: {gcc['monthly']}")
    print(f"    Annual Cost:  {gcc['annual']}")
    print(f"    Break-Even:   {gcc['breakeven']} in Year 1")

# Decision Card
print("\n" + "="*60)
print("📊 Decision Card: Use Tenant Registry When...")
print("="*60)
print("\n✅ YES - Build Registry:")
print("  • 20+ tenants with different requirements")
print("  • Need hierarchical feature flags")
print("  • Compliance requires audit trails")
print("  • Multi-tier service offerings")
print("  • Chargeback/cost attribution needed")

print("\n❌ NO - Use Alternatives:")
print("  • <5 tenants → Manual config files")
print("  • All tenants identical → Single-tenant architecture")
print("  • Budget <₹50L/year → Managed SaaS (Frontegg)")
print("  • <20 tenants + limited customization → Auth0")

# SAVED_SECTION:12

## Section 13: PractaThon Mission & Summary

**From Script Part 3, Section 11: Hands-On Mission**

### PractaThon Mission

Build a tenant registry that:
1. **Manages 5 tenants** with different tiers ✅ (completed above)
2. **Configures tier-specific feature flags** ✅ (completed)
3. **Enforces lifecycle transitions** ✅ (demonstrated)
4. **Demonstrates cascading suspension** ✅ (across 5 systems)
5. **Calculates health scores** ✅ (from mock metrics)

### Validation Tests

Run the test suite:
```bash
pytest tests/test_m11_multi_tenant_foundations.py
```

### What You Built

✅ **PostgreSQL tenant registry** (20+ attributes)
✅ **Feature flag service** (hierarchical evaluation)
✅ **Lifecycle state machine** (valid transitions)
✅ **Cascading operations** (atomic multi-system)
✅ **Health monitoring** framework

### Next Steps

1. Complete PractaThon mission with real database
2. Add audit logging for compliance
3. Implement health score calculation from real metrics
4. Build admin dashboard for tenant management

**Next Module:** M11.3 - Database Multi-Tenancy with Row-Level Security

---

**Great work!** You've built the foundation for production GCC multi-tenancy.

In [ ]:
# Final Summary Statistics
print("="*60)
print("FINAL SUMMARY - Tenant Registry Statistics")
print("="*60)

stats = registry.get_statistics()

print(f"\nTotal Tenants: {stats['total_tenants']}")

print("\nBy Status:")
for status, count in stats['by_status'].items():
    print(f"  {status:15} {count}")

print("\nBy Tier:")
for tier, count in stats['by_tier'].items():
    print(f"  {tier:15} {count}")

print(f"\nAverage Health Score: {stats['average_health_score']}/100")

print("\n" + "="*60)
print("PractaThon Mission: COMPLETE ✅")
print("="*60)
print("\nAccomplishments:")
print("  ✅ Created 5 tenants (1 platinum, 2 gold, 2 silver)")
print("  ✅ Configured tier-specific feature flags")
print("  ✅ Demonstrated lifecycle transitions")
print("  ✅ Showed cascading suspension (5 systems)")
print("  ✅ Calculated tenant health scores")

print("\nNext Steps:")
print("  1. Run test suite: pytest tests/")
print("  2. Start API server: python -m uvicorn app:app --reload")
print("  3. Explore API docs: http://localhost:8000/docs")
print("  4. Next module: M11.3 - Database RLS")

print("\n🎯 Production-Ready Tenant Registry Complete!")

# SAVED_SECTION:13